## Housing Linear Regression

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
import pandas as pd
import os

DATA_PATH = "data"
CSV_FILE = 'Housing.csv'
TEST_SIZE = 0.2
RANDOM_STATE = 0
N_JOBS = -1
SCORING = 'neg_mean_absolute_error'

COLUMN_AREA = 'area'
COLUMN_BEDROOMS = 'bedrooms'
COLUMN_BATHROOMS = 'bathrooms'
COLUMN_PRICE = 'price'
COLUMN_NAMES = [COLUMN_AREA, COLUMN_BEDROOMS, COLUMN_BATHROOMS]
TARGET_COLUMN = COLUMN_PRICE


def load_data(path, filename):
    return pd.read_csv(os.path.join(path, filename)).iloc[:, :4]


def preprocess_data(df):
    X = df[COLUMN_NAMES]
    Y = df[TARGET_COLUMN]
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
    return X_train, X_test, Y_train, Y_test


def train_model(X_train, Y_train):
    model = LinearRegression()
    model.fit(X_train, Y_train)
    return model


def evaluate_model(model, X, Y, X_test, Y_test):
    y_pred = model.predict(X_test)
    n_scores = cross_val_score(model, X, Y, scoring=SCORING, cv=5, n_jobs=N_JOBS)
    n_scores = abs(n_scores)
    mse = mean_squared_error(Y_test, y_pred)
    coef_df = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
    return n_scores, mse, coef_df


df = load_data(DATA_PATH, CSV_FILE)
X_train, X_test, Y_train, Y_test = preprocess_data(df)
model = train_model(X_train, Y_train)
n_scores, mse, coef_df = evaluate_model(model, df[COLUMN_NAMES], df[TARGET_COLUMN], X_test, Y_test)

print('Coefficient:')
print(coef_df)
print('Mean Absolute Error:', n_scores.mean())
print('Mean Squared Error:', mse)

Coefficient:
            Coefficient
area       3.914000e+02
bedrooms   4.745661e+05
bathrooms  1.415260e+06
Mean Absolute Error: 1339111.932553641
Mean Squared Error: 1505858701047.998


## Housing Matrix Factorization Algorithm

In [34]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

def load_data(path, filename):
    return pd.read_csv(os.path.join(path, filename)).iloc[:, :4]

# Load data
data = load_data(DATA_PATH, CSV_FILE)

# We'll use the famous SVD algorithm.
algo = SVD()

# A consumer is needed as a housing buyer but only the rating_scale param is required.
consumer = Reader(rating_scale=(data[COLUMN_PRICE].min(), data[COLUMN_PRICE].max()))

# Prepare the data to be used in Surprise
data[COLUMN_AREA] = data[COLUMN_AREA].astype(str)
data[COLUMN_BEDROOMS] = data[COLUMN_BEDROOMS].astype(str)
data[COLUMN_BATHROOMS] = data[COLUMN_BATHROOMS].astype(str)

# Combine house characteristics to make a unique user for each house
data['user'] = data[[COLUMN_AREA, COLUMN_BEDROOMS, COLUMN_BATHROOMS]].apply(lambda x: '_'.join(x), axis=1)

# Use price as the item (each instance of a price treated as a separate item)
data['item'] = data[COLUMN_PRICE]

# Here we treat area, bedrooms and bathrooms(curated as 'user') as users and price as items
data = Dataset.load_from_df(data[['user', 'item', COLUMN_PRICE]], consumer)

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    7831040.87367274550.09908206922.10018352142.54338788655.13678090662.1505510396.8394
MAE (testset)     7056719.40166340155.55957682085.17897854680.01818474273.48947481582.7295727764.0546
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


{'test_rmse': array([7831040.87356441, 7274550.0989817 , 8206922.10011918,
        8352142.5432877 , 8788655.13670524]),
 'test_mae': array([7056719.40156553, 6340155.55950678, 7682085.178857  ,
        7854680.01809612, 8474273.48939483]),
 'fit_time': (0.0028066635131835938,
  0.002805948257446289,
  0.0025482177734375,
  0.0025708675384521484,
  0.002521991729736328),
 'test_time': (0.0005440711975097656,
  0.00021004676818847656,
  0.0001499652862548828,
  0.00014495849609375,
  0.00014209747314453125)}

### Conclusion
Последний расчет представляет собой нетрадиционное применение матричной факторизации с использованием набора данных о жилье. В этом подходе рассматриваю атрибуты дома (т. е. площадь, спальни и ванные комнаты) как «users», а цены на дома как «items», чтобы сформировать матрицу, которая обрабатывалась бы с использованием алгоритма разложения по сингулярным значениям (SVD) из библиотеки surprise.

Производительность алгоритма SVD оценивалась с использованием среднеквадратической ошибки (RMSE) и средней абсолютной ошибки (MAE) в качестве показателей, а результаты предоставлялись функцией cross_validate. Эти результаты представляют собой 5-кратную перекрестную проверку и будут варьироваться в зависимости от данных.

Однако важно отметить, что такое применение матричной факторизации является нетипичным и может не дать значимой интерпретации в реальном сценарии.